<a href="https://colab.research.google.com/github/melzismn/Digital-Design-2020-2021/blob/master/pca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ONLY FOR COLAB
# Not required in Binder

!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

!conda install -q -y --prefix /usr/local python=3.6 ujson

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

import ujson
print(ujson.dumps({1:2}))

!conda install -c conda-forge igl
!conda install -c conda-forge meshplot

# Import necessary utils

In [ ]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix
import os 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Read and prepare meshes

In [ ]:
# Read and prepare meshes
path_data = "T-Pose-Humans"
onlyfiles = [f for f in os.listdir(path_data) if os.path.isfile(os.path.join(path_data, f))]
for i in range(0,len(onlyfiles)):
  v, f = igl.read_triangle_mesh(os.path.join('.', path_data, str(onlyfiles[i])))
  a = np.reshape(v.T, (-1,1))  
  if i == 0:
    X = a
  else:
    X = np.concatenate((X, a), axis=1)

print("numrows="+str(X.shape[0])+" numcolumns="+str(X.shape[1]))

# Compute and plot the mean shape

In [ ]:
# Compute and plot the mean shape
mean_shape = np.expand_dims(np.mean(X, axis=1),1)
mean_v = np.transpose(np.reshape(mean_shape,(3,-1)))
plot(mean_v, f, shading={"wireframe": True, "colormap": "jet"})

# prepare the covariance matrix and compute the eigendecomposition



In [ ]:
# prepare the covariance matrix and compute the eigendecomposition
X_norm = X - mean_shape
Cov = (1/X_norm.shape[1])*np.matmul(X_norm,X_norm.T)
[Cov.shape[0], Cov.shape[1]]
evals, evecs = np.linalg.eig(Cov)
evals.shape

# Evaluate the eigenvalues

In [ ]:
plt.plot(evals,color='blue', lw=2)
plt.xlabel('evals idx')
plt.ylabel('evals value')
plt.yscale('log')
plt.show()
evals[0:20]

# Generate a new shape


In [ ]:
# fix the basis and generate a random new shape
basis = evecs[:,0:9]
coeff = 0.6*np.random.rand(basis.shape[1],1)
new_shape = np.matmul(basis,coeff)
new_v = mean_v + np.transpose(np.reshape(new_shape,(3,-1)))
plot(new_v, f, shading={"wireframe": True, "colormap": "jet"})
coeff

# analyze one of the training shape

In [ ]:
# analyze one of the input shape from the PCA point of view
i = 4;
in_v, f = igl.read_triangle_mesh(os.path.join('.', path_data, str(onlyfiles[i])))
in_shape = np.reshape(in_v.T, (-1,1))  
vec_in_shape = in_shape - mean_shape 
in_coeff = np.matmul(basis.T, vec_in_shape)
rec_shape = np.matmul(basis,in_coeff)
rec_v = mean_v + np.transpose(np.reshape(rec_shape,(3,-1)))
plot(in_v, f, shading={"wireframe": True, "colormap": "jet"})
plot(rec_v, f, shading={"wireframe": True, "colormap": "jet"})
in_coeff

# Analyze the variation handled by each of the first 4 principal components